In [1]:
import cv2 as cv

import numpy as np

%matplotlib tk
import matplotlib.pyplot as plt


from PIL import Image
import torch

import sys

import matplotlib.pyplot as plt
import os

In [2]:


sys.path.append('./utils')

import model2 as gaze_estimation_module
gaze_estimation_module.load_model()


import model1 as segmentation_module
model_path = "ckpt/sam2.1_hiera_tiny.pt"
SAM2 = "sam2"
segmentation_module.load_model(None, model_path, SAM2)

import image as image_module

import mask as mask_module

# import yolo as yolo_module

load gaze estimator
load the pre-trained model:  ckpt/epoch_24_ckpt.pth.tar


/home/apramey/final-proj/./utils/model2.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pre_trained_model_path, map_location=device)


Loading model
Using SAM2
Model loaded


In [3]:
# Only stereo calcbrate one time
left_frame0 = 'ProjectFPS24/long_video_left_png/frame_1664.png'
right_frame0 = 'ProjectFPS24/long_video_right/frame_1664.jpg'


im_left = cv.imread(left_frame0)
im_left = cv.cvtColor(im_left, cv.COLOR_BGR2RGB) 

im_right = cv.imread(right_frame0)
im_right = cv.cvtColor(im_right, cv.COLOR_BGR2RGB) 

In [4]:
# idea here we need a chessboard to calibrate the camera, world_scaling can be the real size 
rows = 6
cols = 7
world_scaling = 1

In [5]:
def get_chessboard(im, req=42):
    n = 0
    req += 1

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set 42 points to give the chessboard corners')

    clicked = plt.ginput(req, timeout=0, show_clicks=True)
    if not clicked or len(clicked) < req:
        print('Need at least %d lines, you have %d now' % (req, n))


    np_clicked = []
    for i in clicked[1:]:
        np_clicked.append([i])

    np_clicked = np.array(np_clicked)
    return np_clicked

In [6]:
# left_corners = get_chessboard(im_left)
# np.save("left_chessboard",left_corners)


In [7]:
# right_corners = get_chessboard(im_right)
# np.save("right_chessboard", right_corners)

In [8]:
left_corners=np.load("left_chessboard.npy")
left_corners = np.array(left_corners, dtype='float32')

In [9]:
def find_camera_matrix_auto(image):
    objp = np.zeros((rows*cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:cols].T.reshape(-1,2)
    objp = objp * world_scaling
    width = image.shape[1]
    height = image.shape[0]   

    img_point = []
    points_3d = []
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    ret, corners = cv.findChessboardCorners(gray, (rows,cols), cv.CALIB_CB_NORMALIZE_IMAGE)
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        img_point.append(corners2)
        points_3d.append(objp)

        # cv.drawChessboardCorners(image, (rows,cols), corners2, ret)
        # cv.imshow('image', image)
        # cv.waitKey(0)
        # cv.destroyAllWindows()
    else:
        print('corners not found')
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(points_3d,img_point, (width, height), None, None)
    print(ret)
    return mtx, dist, img_point, points_3d

def find_camera_matrix_manual(image, corners):
    objp = np.zeros((rows*cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:cols].T.reshape(-1,2)
    objp = objp * world_scaling
    width = image.shape[1]
    height = image.shape[0]   

    img_point = []
    points_3d = []
    img_point.append(corners)
    points_3d.append(objp)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # cv.imshow('image', image)
    # cv.waitKey(0)
    # cv.destroyAllWindows()


    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(points_3d,img_point, (width, height), None, None)
    print(ret)
    return mtx, dist, img_point, points_3d


In [10]:


mtx_R, dist_R, img_point_R, points_3d_R = find_camera_matrix_auto(im_right)

mtx_L, dist_L, img_point_L, points_3d_L =  find_camera_matrix_manual(im_left, left_corners)


0.15130642723709312
0.41953839100710766


In [11]:

# points = []
# for i in img_point_R[0]:
#     points.append(i[0])
# points = np.array(points[:7])
# print(points)
# plt.figure()
# plt.imshow(im_right)
# plt.scatter(points[:,0], points[:,1])

In [12]:
print(dist_L)
print(dist_R)


width = im_left.shape[1]
height = im_left.shape[0]
print(width, height)
width = im_right.shape[1]
height = im_right.shape[0]
print(width, height)

[[-0.22851414  0.21614433  0.01018155 -0.012397    0.15947108]]
[[-7.48070944e+00  1.87660564e+02 -2.67514700e-01  9.34490885e-03
  -1.89821993e+03]]
1920 1080
3840 2160


In [13]:
def find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d):
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
    ret, CM1, dist1, CM2, dist2, R, T, E, F = cv.stereoCalibrate(points_3d, img_point_L, img_point_R, mtx_L, dist_L,
                                mtx_R, dist_R, (width, height), criteria = criteria, flags = stereocalibration_flags)
    return R, T


In [14]:
R,T = find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d_L)

In [15]:
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
P1 = mtx_L @ RT1

RT2 = np.concatenate([R, T], axis = -1)
P2 = mtx_R @ RT2


In [16]:
def find_center(M, shown = False):
    U, S, V = np.linalg.svd(M)
    if shown:
        print(S)
        print("Camera Center:", V[-1, :3] / V[-1, -1])
    return V[-1, :3] / V[-1, -1]

In [17]:
def DLT(P1, P2, point1, point2):
 
    A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
    A = np.array(A).reshape((4,4))
    #print('A: ')
    #print(A)
 
    B = A.transpose() @ A
    from scipy import linalg
    U, s, Vh = linalg.svd(B, full_matrices = False)
 
    # print('Triangulated point: ')
    # print(Vh[3,0:3]/Vh[3,3])
    return Vh[3,0:3]/Vh[3,3]

In [18]:
pt = DLT(P1, P2, img_point_L[0][0][0], img_point_R[0][0][0])
print(pt)

[-58.8905832  -18.87801782 128.99025931]


In [19]:
pt = DLT(P1, P2, img_point_L[0][1][0], img_point_R[0][1][0])

In [20]:
c1 = find_center(P1)
c2 = find_center(P2)

print(c1)
print(c2)


[0. 0. 0.]
[66.7928762   3.35899459 -0.07095524]


In [21]:
def get_input_lines(im, min_lines=4):
    n = 0

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set 4 points to give the plane of table')

    clicked = plt.ginput(min_lines, timeout=0, show_clicks=True)
    if not clicked or len(clicked) < min_lines:
        print('Need at least %d lines, you have %d now' % (min_lines, n))


    
    pt1 = np.array([clicked[0][0], clicked[0][1], 1])
    pt2 = np.array([clicked[1][0], clicked[1][1], 1])
    pt3 = np.array([clicked[2][0], clicked[2][1], 1])
    pt4 = np.array([clicked[3][0], clicked[3][1], 1])

    return [pt1, pt2, pt3, pt4]

In [22]:
# pts_left = (get_input_lines(im_left, 4))
# pts_right = (get_input_lines(im_right, 4))

In [23]:
# np.save("pts_left", pts_left)
# np.save("pts_right", pts_right)


In [24]:
pts_left = np.load("pts_left.npy")
pts_right = np.load("pts_right.npy")

In [25]:
pts_x = []
pts_y = []
pts_z = []

for i in range(img_point_L[0].shape[0]):
    pt = DLT(P1, P2, img_point_L[0][i][0], img_point_R[0][i][0])
    # pt = pt / pt[2]
    pts_x.append(pt[0])
    pts_y.append(pt[1])
    pts_z.append(pt[2])
print(len(pts_x), len(pts_y), len(pts_z))


42 42 42


In [26]:
pt_table = np.array([DLT(P1, P2, pts_left[0], pts_right[0]), DLT(P1, P2, pts_left[1], pts_right[1]), DLT(P1, P2, pts_left[2], pts_right[2])])

v1 = pt_table[0] - pt_table[1]
v2 = pt_table[0] - pt_table[2]
normal = np.cross(v1, v2)

d = -np.dot(normal, pt_table[0])



In [27]:
fig = plt.figure()

ax_3d = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax_3d.view_init(elev=135, azim=0)
ax_3d.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax_3d.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax_3d.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax_3d.scatter(pts_x, pts_y, pts_z, c='b')  
ax_3d.scatter(c1[0], c1[1], c1[2], c='r')
ax_3d.scatter(c2[0], c2[1], c2[2], c='r')
ax_3d.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g')
ax_3d.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g')
ax_3d.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
ax_3d.legend(['Chessboard', 'Camera 1', 'Camera 2', 'Table'])
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax_3d.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
plt.show()

In [28]:

def find_plane_two_vectors(c, d1, d2):
    """
    Find the plane passing through the camera center and spanned by two direction vectors.

    Parameters:
        c (numpy.ndarray): Camera center coordinates (3D point).
        d1 (numpy.ndarray): First direction vector (3D).
        d2 (numpy.ndarray): Second direction vector (3D).

    Returns:
        n (numpy.ndarray): Normalized normal vector of the plane.
        d (float): Plane constant such that n . x = d for any point x on the plane.
    """
    # Step 1: Compute the normal vector to the plane
    n = np.cross(d1, d2)  # Cross product gives the perpendicular vector
    if np.linalg.norm(n) == 0:
        raise ValueError("Direction vectors are collinear; no unique plane can be defined.")
    
    n = n / np.linalg.norm(n)  # Normalize the normal vector
    # Step 2: Compute the plane constant 'd' using the camera center 'c'
    d = -np.dot(n, c)  # Plane equation: n . x + d = 0

    return n, d

In [29]:

def backproject(P, x):
    """
    Backproject a 2D image point x into a 3D ray given a camera projection matrix P.

    Parameters:
        P (numpy.ndarray): 3x4 camera projection matrix.
        x (numpy.ndarray): 2D image point in homogeneous coordinates [u, v, 1].
    
    Returns:
        d (numpy.ndarray): 3D ray direction vector.
    """

    # Step 2: Construct the backprojected ray direction
    # Solve for a point along the ray: X = [X, Y, Z, 1] such that x = P * X
    # Assume depth Z=1 for simplicity
    x_h = np.array([x[0], x[1], 1])  # Make sure x is in homogeneous form
    M = P[:, :3]  # Extract the camera intrinsic matrix
    d = np.linalg.inv(M) @ x_h  # Backproject the point
    d = d / np.linalg.norm(d)  # Normalize the direction vector
    return d



In [30]:
def intersection_line_of_planes(n1, d1, n2, d2):
    """
    Find the intersection line of two planes in 3D.

    Parameters:
        n1 (numpy.ndarray): Normal vector of plane 1.
        d1 (float): Plane 1 constant.
        n2 (numpy.ndarray): Normal vector of plane 2.
        d2 (float): Plane 2 constant.

    Returns:
        X0 (numpy.ndarray): A point on the intersection line.
        d (numpy.ndarray): Direction vector of the intersection line.
    """
    # Step 1: Direction of the intersection line (cross product of normals)
    d =  np.cross(n1, n2)
    
    # Check if the planes are parallel
    if np.linalg.norm(d) == 0:
        raise ValueError("Planes are parallel, no intersection line.")
    
    # Step 2: Find a point on the intersection line
    # Solve for a point X0 satisfying both plane equations
    A = np.vstack((n1, n2, d))  # Stack normal vectors and direction vector
    b = np.array([d1, d2, 0])   # Right-hand side with 0 for d direction
    
    # Solve the linear system for X0
    X0 = - np.linalg.lstsq(A, b, rcond=None)[0]
    
    return X0, d


In [31]:
# Gaze estimation on all the frames

In [32]:
img, small = image_module.load_img(left_frame0)
image_module.set_img(small)

print(img.shape)
# gaze estimation

arrow_start_left, arrow_end_left = gaze_estimation_module.find_vector_arrow(img)

(1080, 1920, 3)
detected_faces:  1
pred_gaze_np:  [-0.04297274 -0.3647134 ]


In [33]:
plt.figure()
plt.imshow(img)
plt.scatter(arrow_start_left[0], arrow_start_left[1], c='r')
plt.show()

In [34]:
img, small = image_module.load_img(right_frame0)

arrow_start_right, arrow_end_right = gaze_estimation_module.find_vector_arrow(img)

detected_faces:  1
pred_gaze_np:  [-0.41596723 -0.1681314 ]


In [35]:
plt.figure()
plt.imshow(img)
plt.scatter(arrow_start_right[0], arrow_start_right[1], c='r')
plt.show()


In [36]:
face_3d = DLT(P1, P2, arrow_start_left, arrow_start_right)

In [37]:
v_left_1 = backproject(P1, arrow_start_left)
v_left_2 = backproject(P1, arrow_end_left)

v_right_1 = backproject(P2, arrow_start_right)
v_right_2 = backproject(P2, arrow_end_right)

n_left, d_left = find_plane_two_vectors(face_3d, v_left_1, v_left_2)
n_right, d_right = find_plane_two_vectors(c2, v_right_1, v_right_2)

c_, d_line = intersection_line_of_planes(n_left, d_left, n_right, d_right)



In [38]:
fig = plt.figure()

ax_3d = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax_3d.view_init(elev=135, azim=0)
ax_3d.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax_3d.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax_3d.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax_3d.scatter(pts_x, pts_y, pts_z, c='b', label='Chessboard')
ax_3d.scatter(c1[0], c1[1], c1[2], c='r', label='Camera 1')
ax_3d.scatter(c2[0], c2[1], c2[2], c='purple',  label='Camera 2')
ax_3d.scatter(face_3d[0], face_3d[1], face_3d[2], c='black', label='Face')
ax_3d.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g', label='Table')
ax_3d.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g' )
ax_3d.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax_3d.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
t = np.linspace(-200, 200, 100)


X = c_[0] + t * d_line[0]
Y = c_[1] + t * d_line[1]
Z = c_[2] + t * d_line[2]


ax_3d.plot(X, Y, Z, c='orange', label='Gaze vector')
ax_3d.legend()
plt.show()

In [39]:
# def get_masks_on_table(frame_path, resize_scale=0.5):
#     img, small = image_module.load_img(frame_path, resize_scale)
#     image_module.set_img(img)

#     # gaze estimation
#     arrow_start, arrow_end = gaze_estimation_module.find_vector_arrow(img)

#     # yolo implementation (finds table)
#     table_img, curr_table = yolo_module.get_table(small)
#     reset_table_bbox = torch.tensor([0, 0, curr_table[2]-curr_table[0], curr_table[3]-curr_table[1]])

#     #segmentation
#     SAM2 = "sam2"
#     masks = segmentation_module.mask_generate(table_img, SAM2)
#     masks_on_table = mask_module.find_object_given_table(reset_table_bbox, masks)

#     new_masks = []

#     for mask in masks_on_table:
#         seg = mask["segmentation"]

#         full_size_seg = np.zeros(img.shape[:-1], dtype=bool)
#         full_size_seg[int(curr_table[1]):int(curr_table[3])+1,int(curr_table[0]):int(curr_table[2])+1] = seg

#         mask['segmentation'] = full_size_seg
        

#     return masks_on_table

In [40]:
def get_masks_on_table(frame_path, is_left, resize_scale=0.5):
    img, small = image_module.load_img(frame_path, resize_scale)
    image_module.set_img(img)

    # gaze estimation
    arrow_start, arrow_end = gaze_estimation_module.find_vector_arrow(img)

    # Get table bounding box from points left
    # We have three corners
    if is_left:
        corner1 = pts_left[0][:2]
        corner2 = pts_left[1][:2]
        corner3 = pts_left[2][:2]
        corner4 = pts_left[3][:2]
    else:
        corner1 = pts_right[0][:2]
        corner2 = pts_right[1][:2]
        corner3 = pts_right[2][:2]
        corner4 = pts_right[3][:2]

    xs = np.array([corner1[1], corner2[1], corner3[1], corner4[1]])
    ys = np.array([corner1[0], corner2[0], corner3[0], corner4[0]])
    max_x = np.max(xs)
    max_y = np.max(ys)

    min_x = np.min(xs)
    min_y = np.min(ys)


    table_bbox = np.array([min_y, min_x,max_y - min_y, max_x - min_x])


    #segment the whole table
    SAM2 = "sam2"
    masks = segmentation_module.mask_generate(img, SAM2)
    # image_module.show_box(masks[7]['bbox'])
    masks_on_table = mask_module.find_object_given_table(table_bbox, masks)


    return masks_on_table

In [41]:
left_frame_masks = get_masks_on_table(left_frame0, True)

image_module.set_img(image_module.load_img(left_frame0)[0])
image_module.show_anns(left_frame_masks)
plt.show()

right_frame_masks = get_masks_on_table(right_frame0, False)

image_module.set_img(image_module.load_img(right_frame0)[0])
image_module.show_anns(right_frame_masks)
plt.show()

detected_faces:  1
pred_gaze_np:  [-0.04297274 -0.3647134 ]
Generating mask
detected_faces:  1
pred_gaze_np:  [-0.41596723 -0.1681314 ]
Generating mask


In [42]:
def reconstruct_3d(masks_left, masks_right):
    pt_mask_right = []
    pt_mask_left = []
    for mask in masks_left:
        y_coords, x_coords = np.where(mask["segmentation"])
        x = np.mean(x_coords)
        y = np.mean(y_coords)
        pt_mask_left.append([x, y])

    for mask in masks_right:
        y_coords, x_coords = np.where(mask["segmentation"])
        x = np.mean(x_coords)
        y = np.mean(y_coords)
        pt_mask_right.append([x, y])

    pt_mask_3d = []
    indexs = []
    for i in range(len(pt_mask_left)):
        min_dis = 1000000
        cur_pt = None
        index = -1
        for j in range(len(pt_mask_right)):
            pt = DLT(P1, P2, pt_mask_left[i], pt_mask_right[j])
            dist = abs(np.dot(normal, pt ) + d)
            if dist < min_dis:
                min_dis = dist
                cur_pt = pt
                index = j
            # pt_mask_3d.append(pt)
        pt_mask_3d.append(cur_pt)
        indexs.append(index)
    return np.array(pt_mask_3d), np.array(indexs)

In [43]:
fig = plt.figure()

ax_3d = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax_3d.view_init(elev=135, azim=0)
ax_3d.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax_3d.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax_3d.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax_3d.scatter(pts_x, pts_y, pts_z, c='b', label='Chessboard')
ax_3d.scatter(c1[0], c1[1], c1[2], c='r', label='Camera 1')
ax_3d.scatter(c2[0], c2[1], c2[2], c='purple',  label='Camera 2')
ax_3d.scatter(face_3d[0], face_3d[1], face_3d[2], c='gold', label='Face')
ax_3d.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g', label='Table')
ax_3d.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g' )
ax_3d.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax_3d.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
t = np.linspace(-200, 200, 100)


X = c_[0] + t * d_line[0]
Y = c_[1] + t * d_line[1]
Z = c_[2] + t * d_line[2]


ax_3d.plot(X, Y, Z, c='orange', label='Gaze vector')









################################################################################
pointsd, indexs = reconstruct_3d(left_frame_masks, right_frame_masks)

ax_3d.scatter(pointsd[:, 0], pointsd[:, 1], pointsd[:,2], c='black', label='Reconstructed points')

ax_3d.legend()
plt.show()

In [44]:
pointsd, indexs = reconstruct_3d(left_frame_masks, right_frame_masks)

# ax_3d.scatter(pointsd[:, 0], pointsd[:, 1], pointsd[:,2], c='black', label='Reconstructed points')

# ax_3d.legend()
# plt.show()

In [45]:
# plt.imshow(image_module.load_img(left_frame0)[0])
# plt.show()

In [46]:
# plt.imshow(image_module.load_img(right_frame0)[0])

In [47]:
def find_closest_point(points, x0, d):
    # x0: intercept point on the line, a vector [x0_x, x0_y, x0_z]
    # d: direction vector of the line [d_x, d_y, d_z]
    # points: list of points [[x1, y1, z1], [x2, y2, z2], ...]
    
    closest_point = None
    index = -1
    min_dist_sq = float('inf')
    
    dd = d[0]*d[0] + d[1]*d[1] + d[2]*d[2]  # d·d
    
    for i in range(len(points)):
        p = points[i]
        px, py, pz = p
        t = ((px - x0[0])*d[0] + (py - x0[1])*d[1] + (pz - x0[2])*d[2]) / dd

        proj = [x0[0] + t*d[0], x0[1] + t*d[1], x0[2] + t*d[2]]
        
        dx = p[0] - proj[0]
        dy = p[1] - proj[1]
        dz = p[2] - proj[2]
        dist_sq = dx*dx + dy*dy + dz*dz

        if dist_sq < min_dist_sq:
            min_dist_sq = dist_sq
            closest_point = p
            index = i
    
    return closest_point, index, min_dist_sq


In [48]:
closest_point, index, mds = find_closest_point(pointsd, c_, d_line)
# pointsd.pop(index)
# second_closest_point, index2 = find_closest_point(pointsd, c_, d_line)

In [49]:
left_frame_masks[index]

{'segmentation': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 'area': 1968,
 'bbox': [1087.0, 675.0, 69.0, 40.0],
 'predicted_iou': 0.9593369364738464,
 'point_coords': [[1110.0, 691.875]],
 'stability_score': 0.9733668565750122,
 'crop_box': [0.0, 0.0, 1920.0, 1080.0]}

In [50]:
left_list = os.listdir('ProjectFPS24/long_video_left_png')
left_list.sort(key=lambda x: x[6:10])
left_list = left_list[1653:-9]

right_list = os.listdir('ProjectFPS24/long_video_right')
right_list.sort(key=lambda x:x[6:10])

In [53]:
for i in range(len(left_list)):
    left_frame = os.path.join('ProjectFPS24/long_video_left_png/', f"{left_list[i]}")
    right_frame = os.path.join('ProjectFPS24/long_video_right/', f"{right_list[i]}")
    if i == 0:
        left_frame_masks = get_masks_on_table(left_frame, True)
        right_frame_masks = get_masks_on_table(right_frame, False)

        pointsd, indexs = reconstruct_3d(left_frame_masks, right_frame_masks)


    im_left, small_left = image_module.load_img(left_frame)
    arrow_start_left, arrow_end_left = gaze_estimation_module.find_vector_arrow(im_left.copy())



    im_right, small_right = image_module.load_img(right_frame)
    arrow_start_right, arrow_end_right = gaze_estimation_module.find_vector_arrow(im_right.copy())


    face_3d = DLT(P1, P2, arrow_start_left, arrow_start_right)

    v_left_1 = backproject(P1, arrow_start_left)
    v_left_2 = backproject(P1, arrow_end_left)

    v_right_1 = backproject(P2, arrow_start_right)
    v_right_2 = backproject(P2, arrow_end_right)

    n_left, d_left = find_plane_two_vectors(face_3d, v_left_1, v_left_2)
    n_right, d_right = find_plane_two_vectors(c2, v_right_1, v_right_2)

    c_, d_line = intersection_line_of_planes(n_left, d_left, n_right, d_right)




    closest_point, index, mds = find_closest_point(pointsd, c_, d_line)

    image_module.set_img(im_left)
    print(index)
    image_module.show_box(left_frame_masks[index]["bbox"], out_path=f'long_video_out/frame_{i:04}')

    image_module.set_img(im_right)
    image_module.show_box(right_frame_masks[indexs[index]]["bbox"])


detected_faces:  1
pred_gaze_np:  [ 0.02944824 -0.15444033]
Generating mask
detected_faces:  1
pred_gaze_np:  [-0.41596723 -0.1681314 ]
Generating mask
detected_faces:  1
pred_gaze_np:  [ 0.02944824 -0.15444033]
detected_faces:  1
pred_gaze_np:  [-0.41596723 -0.1681314 ]
5
detected_faces:  1
pred_gaze_np:  [ 0.02641964 -0.12324595]
detected_faces:  1
pred_gaze_np:  [-0.50371486 -0.18448395]
5
detected_faces:  1
pred_gaze_np:  [ 0.02369107 -0.14577402]
detected_faces:  1
pred_gaze_np:  [-0.4920427  -0.17034237]
5
detected_faces:  1
pred_gaze_np:  [-0.04224747 -0.21444286]
detected_faces:  1
pred_gaze_np:  [-0.5560435  -0.12618935]
2
detected_faces:  1
pred_gaze_np:  [-0.01626143 -0.18067637]
detected_faces:  1
pred_gaze_np:  [-0.5436598  -0.14569414]
5
detected_faces:  1
pred_gaze_np:  [-0.10642676 -0.16047259]
detected_faces:  1
pred_gaze_np:  [-0.56892383 -0.07619727]
2
detected_faces:  1
pred_gaze_np:  [-0.1117917  -0.17445567]
detected_faces:  1
pred_gaze_np:  [-0.5900352  -0.204850

/home/apramey/final-proj/./utils/image.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10,10))


detected_faces:  1
pred_gaze_np:  [-0.07007199 -0.36699638]
detected_faces:  1
pred_gaze_np:  [-0.6271393   0.42653325]
5
detected_faces:  1
pred_gaze_np:  [-0.09720109 -0.35949996]
detected_faces:  1
pred_gaze_np:  [-0.6579491   0.47391677]
5
detected_faces:  1
pred_gaze_np:  [-0.18221647 -0.23029993]
detected_faces:  1
pred_gaze_np:  [-0.6075465   0.48065928]
3
detected_faces:  1
pred_gaze_np:  [-0.18132925 -0.20404841]
detected_faces:  1
pred_gaze_np:  [-0.59777343  0.35453987]
11
detected_faces:  1
pred_gaze_np:  [-0.18892194 -0.2132721 ]
detected_faces:  1
pred_gaze_np:  [-0.5407214  0.3585325]
11
detected_faces:  1
pred_gaze_np:  [-0.20376436 -0.23886147]
detected_faces:  1
pred_gaze_np:  [-0.5730104   0.31714815]
11
detected_faces:  1
pred_gaze_np:  [-0.34007683 -0.21613519]
detected_faces:  1
pred_gaze_np:  [-0.5291611   0.80492324]
10
detected_faces:  1
pred_gaze_np:  [-0.3016677 -0.3150484]
detected_faces:  1
pred_gaze_np:  [-0.44870174  0.88875365]
3
detected_faces:  1
pred_

NameError: name 'exit' is not defined